In [ ]:
import pandas as pd

## Задание №1
Загрузить файл text.txt c заданиями, руками привести существительные в скобках в форму родительного падежа. <br>
Сохранить результаты в файл задание1.txt в формате:  <br> 
Купить килограмм - {слово в родительном падеже}

In [ ]:
with open(r'./data/text.txt', encoding="utf-8") as f:
    lines = f.readlines()

In [ ]:
list_text = lines[0].split('),')

In [ ]:
list_task=[]
for el in list_text:
    list_el = []
    temp_el = el.split('(')
    list_el.append(temp_el[0])
    for word_task in temp_el[1].split(','): list_el.append(word_task.replace(' ',''))
    list_task.append(list_el)
    

In [ ]:
assert len(list_task) == 32

In [ ]:
#Для удосбтва можно воспользоваться кодом ниже, данный код для одного задания
unsver_list = []
contexst = list_task[0][0]
for word in list_task[0][1:]:
    
    unsver = input(f'Форма слова - {word} в родительном падеже в контексте {contexst}')
    unsver_list.append([contexst + unsver])


In [ ]:
#all_unsver_list
with open("задание1.txt", "w") as output:
    output.write(str(all_unsver_list))

## Задание №2
Воспользоваться уже сформированным list_task. Воспользуйтесь словарём сформированным из Викисловаря повторите задание №1 <br>
Для слов которых нет в словаре, попробуйте создать сами конструкцию из if-ов которые переведут слово в родительный падаж.<br>
Для более точного подхода можно воспользоваться https://ru.wiktionary.org/wiki/Викисловарь:Шаблоны_словоизменений/Существительные <br>

Сохраните результаты в файл задание2.txt в формате:  <br> 
Купить килограмм - {слово в родительном падеже}

In [ ]:
df = pd.read_csv(r'./data/data_lem.csv', index_col=0)
df.head()

## Задание №3
Воспользоваться уже сформированным list_task. Воспользуйтесь библиотекой pymorphy2 и повторите задание №1 <br>
Для слов которых нет в словаре создайте конструкцию из if-ов которые переведут слово в родительный падеж <br>
Сохраните результаты в файл задание3.txt в формате:  <br> 
Купить килограмм - {слово в родительном падеже}